loop epoch

-   loop batches

is preferable for large dataset

Epoch = 1 forward and backward pass of ALL training samples

batch_size = number of training samples in one forward & backward pass

number of iterations = number of passes, each pass using [batch_size] number of samples

e.g. 100 samples, batch_size=20 -> 100/20 = 5 iterations for 1 epoch

In [1]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch.nn as nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
class WineTrain(Dataset):

    def __init__(self):
        # data loading
        # using , to split the data, skip the row 1 because it is the title
        xy = np.loadtxt("/home/tian/Projects/d2l/data/wine.csv", delimiter=",", dtype=np.float32, skiprows=1)
        # column 0 is the classes, the input should be those features in column[1:]
        x = torch.from_numpy(xy [:, 1:].astype(np.float32))
        y = torch.from_numpy(xy [:, [0]].astype(np.float32)) # size = n_sample, 1
        self.n_labels = int(max(y.squeeze(1).numpy()))
        sc = MinMaxScaler()
        self.x, _, self.y, _ = train_test_split(x, y, test_size=0.2, random_state=1)
        self.x = sc.fit_transform(self.x)
        self.x = torch.from_numpy(self.x.astype(np.float32))
        # self.y = self.y.type(torch.int64)
        # self.y = nn.functional.one_hot(self.y , -1)
        # self.y = self.y.squeeze(1).type(torch.float32)
        self.y = self.y.squeeze(1)- 1
        self.n_features = self.x.shape[1]
        self.n_samples = self.x.shape[0]



    def __getitem__(self, index):
        # dataset[0]
        return self.x[index], self.y[index]

    def __len__(self):
        # len(dataset)
        return self.n_samples


In [3]:
train_data = WineTrain()

In [14]:
class WineTest(Dataset):

    def __init__(self):
        # data loading
        # using , to split the data, skip the row 1 because it is the title
        xy = np.loadtxt("/home/tian/Projects/d2l/data/wine.csv", delimiter=",", dtype=np.float32, skiprows=1)
        # column 0 is the classes, the input should be those features in column[1:]
        x = torch.from_numpy(xy [:, 1:].astype(np.float32))
        y = torch.from_numpy(xy [:, [0]].astype(np.float32)) # size = n_sample, 1
        self.n_labels = int(max(y.squeeze(1).numpy()))
        sc = MinMaxScaler()
        _, self.x, _, self.y = train_test_split(x, y, test_size=0.2, random_state=1)
        self.x = sc.fit_transform(self.x)
        self.x = torch.from_numpy(self.x.astype(np.float32))
        # self.y = torch.from_numpy(self.y.astype(np.float32))
        self.y = self.y.squeeze(1)- 1
        self.n_features = self.x.shape[1]
        self.n_samples = self.x.shape[0]


    def __getitem__(self, index):
        # dataset[0]
        return self.x[index], self.y[index]

    def __len__(self):
        # len(dataset)
        return self.n_samples

In [15]:
test_data = WineTest()

In [6]:
# train_data = WineTrain()
# test_data = WineTest()
batch_size = 10
dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, num_workers=2)

In [7]:
# play with Dataset

first_data = train_data[0]
features, labels = first_data
features, labels

(tensor([0.2553, 0.1245, 0.5668, 0.6369, 0.1739, 0.1621, 0.1920, 0.7400, 0.3849,
         0.1980, 0.4310, 0.5055, 0.1227]),
 tensor(1.))

In [8]:
# # play with dataloader

# datatiter = iter(dataloader)
# data = next(datatiter)
# features, labels = data
# features.shape, labels.shape

In [9]:
# 1 model
# f = wx + b, sigmoid at the end


class Model(nn.Module):

    def __init__(self, n_features, n_labels):
        super(Model, self).__init__()
        self.l1 = nn.Linear(n_features, 64)
        self.sm = nn.Sigmoid()
        self.l2 = nn.Linear(64, n_labels)

    
    def forward(self, x):
        x = self.sm(self.l1(x))
        x = self.l2(x)
        return x

model = Model(n_features=train_data.n_features, n_labels=train_data.n_labels)
# model.to(device)

In [ ]:
print('W:',list(model.parameters())[0].size())
print('b',list(model.parameters())[1].size())

In [10]:
# training hyperpara
total_samples = len(train_data)
n_iterations = math.ceil(total_samples/batch_size)   # have the int
total_samples, n_iterations

(142, 15)

In [11]:
num_epochs = 100
lr = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [12]:
# training loop

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        # inputs = inputs.to(device)
        labels = labels.type(torch.LongTensor)
        # forward backward, update
        y_pred = model(inputs)
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % batch_size ==0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_iterations}, inputs {inputs.shape}, loss = {loss.item():.4f}')



epoch 1/100, step 10/15, inputs torch.Size([10, 13]), loss = 1.1771
epoch 2/100, step 10/15, inputs torch.Size([10, 13]), loss = 0.8645
epoch 3/100, step 10/15, inputs torch.Size([10, 13]), loss = 0.7937
epoch 4/100, step 10/15, inputs torch.Size([10, 13]), loss = 0.7202
epoch 5/100, step 10/15, inputs torch.Size([10, 13]), loss = 0.4578
epoch 6/100, step 10/15, inputs torch.Size([10, 13]), loss = 0.4133
epoch 7/100, step 10/15, inputs torch.Size([10, 13]), loss = 0.3043
epoch 8/100, step 10/15, inputs torch.Size([10, 13]), loss = 0.2113
epoch 9/100, step 10/15, inputs torch.Size([10, 13]), loss = 0.1367
epoch 10/100, step 10/15, inputs torch.Size([10, 13]), loss = 0.2117
epoch 11/100, step 10/15, inputs torch.Size([10, 13]), loss = 0.2583
epoch 12/100, step 10/15, inputs torch.Size([10, 13]), loss = 0.1250
epoch 13/100, step 10/15, inputs torch.Size([10, 13]), loss = 0.0591
epoch 14/100, step 10/15, inputs torch.Size([10, 13]), loss = 0.1019
epoch 15/100, step 10/15, inputs torch.Size

In [27]:
with torch.no_grad():
    y_pred = model(test_data.x)
    print(y_pred[:5])
    y_pred_cls = y_pred.round()
    print(y_pred_cls[:5])
    y_pred_cls_one = torch.argmax(y_pred_cls, dim=1)
    print(y_pred_cls_one[:5])
    acc = y_pred_cls_one.eq(test_data.y).sum() / float(test_data.n_samples)
    print(f'accuracy = {acc:.4f}')

tensor([[ -1.7424,  -3.9385,   3.7010],
        [ -4.3664,  12.2377, -12.4058],
        [  8.2782,  -2.2371,  -7.9188],
        [ -2.2519,  10.9694, -13.0374],
        [  9.5574,  -4.6117,  -6.2853]])
tensor([[ -2.,  -4.,   4.],
        [ -4.,  12., -12.],
        [  8.,  -2.,  -8.],
        [ -2.,  11., -13.],
        [ 10.,  -5.,  -6.]])
tensor([2, 1, 0, 1, 0])
accuracy = 0.9722
